# **SysML v2 Language Demonstration<br/>Modeling in Time and Space**
# *INCOSE International Symposium</br>July 2023*

# Ed Seidewitz
© Copyright 2023 Model Driven Solutions, Inc.

# Vehicle Definitions

In [1]:
package VehicleDefinitions {
    import SI::*;
    import Time::*;
    
    action def SenseVehicleTemperature {
        out temp :> ISQ::temperature;
    }
    state def VehicleHealthStates;
    
    part def Vehicle;   
    part def VehicleController;
    
    item def OverTempSignal {
        attribute temp :> ISQ::temperature;
    }
    item def ReturnToNormalSignal;

}

Package VehicleDefinitions (ed248a6a-971a-406b-8bcd-656ebbfbdd7c)


# Vehicle Configuration

In [3]:
package VehicleConfiguration {
    import VehicleDefinitions::*;
    
    part vehicle : Vehicle {
        attribute maxTemp :> ISQ::temperature;
        attribute maintenanceTime :> Time::timeInstant;
        attribute maintenanceInterval :> ISQ::duration default 90 * 24 [h];
        attribute maintenanceDuration :> ISQ::duration default 48 [h];
        
        part controller : VehicleController;
        
        state healthStates : VehicleHealthStates {
            entry; then normal;
            do action senseTemperature : SenseVehicleTemperature;

            state normal;

            transition normal_maintenance
                first normal
                accept at maintenanceTime
                then maintenance;
            
            state maintenance {
                assert constraint { TimeOf(maintenance) > maintenanceTime }
                assert constraint { TimeOf(maintenance) - TimeOf(normal.done) < 2 [s] }
                do assign maintenanceTime := maintenanceTime + maintenanceInterval;
            }
            
            constraint maintenanceDurationConstraint { 
                DurationOf(maintenance) <= maintenanceDuration 
            }

            transition normal_degraded
                first normal
                accept when senseTemperature.temp > maxTemp
                do action sendSignal send OverTempSignal(senseTemperature.temp) to controller
                then degraded;

            state degraded;
            
            transition maintenance_normal
                first maintenance
                accept ReturnToNormalSignal
                then normal;

            transition degraded_normal
                first degraded
                accept ReturnToNormalSignal
                then normal;
        }
    }
}

Package VehicleConfiguration (be1bff1d-5947-480d-9035-2b32003782c3)


# Vehicle Scenario

In [4]:
package VehicleScenario {
    import VehicleConfiguration::*;
    
    individual part def Vehicle1 :> Vehicle;
    individual part vehicle1 : Vehicle1 :> vehicle {
        attribute startTime = Iso8601DateTime("2022-03-22T09:00:00-04:00");        
        attribute :>> maxTemp = 90 ['°C'];
        
        timeslice vehicleMaintenance[1] {
            assert constraint { TimeOf(vehicleMaintenance.start) == startTime }
            
            attribute :>> maintenanceTime := startTime;
            
            state :>> healthStates : VehicleHealthStates[1] {
                state :>> normal[1] {
                    assert constraint { TimeOf(normal.done) == startTime }
                }
                then state :>> maintenance[1] {
                    assert constraint { TimeOf(maintenance.start) == startTime + 2 [s] }
                    assert constraint { TimeOf(maintenance.done) == startTime + 24 [h] }
                    assert maintenanceDurationConstraint;
                }
            }
        }
        
        then timeslice vehicleDegraded[1] {           
            state :>> healthStates : VehicleHealthStates[1] {
                do action :>> senseTemperature[1] {
                    out :>> temp = maxTemp + 10 ['°C'];
                }
                
                state :>> normal[1];
                then perform normal_degraded.sendSignal;
                then state :>> degraded[1];
            }
        }
    }     
}

Package VehicleScenario (a239090f-8109-4c78-8429-3d004125fbcf)


# Vehicle Geometry

In [5]:
package VehicleGeometry {
    import VehicleDefinitions::*;
    import ShapeItems::*;
    import SpatialItems::*;
    import MeasurementReferences::CoordinateFramePlacement;
    
    part def Wheel :> SpatialItem {
        item :>> shape : Cylinder {
            :>> radius = 70 [cm];
            :>> height = 22 [cm];
        }        
    }
    
    part def Chassis :> SpatialItem {
        item :> envelopingShapes : Box[1] {
            :>> length = 480 [cm];
            :>> width = 184 [cm];
            :>> height = 135 [cm];
        }
    }
    
    part vehicle : Vehicle, SpatialItem {        
        attribute vcf :>> coordinateFrame {
            :>> mRefs = (cm, cm, cm);
        }
        
        part chassis : Chassis[1] :> componentItems {
            attribute :>> coordinateFrame : CoordinateFramePlacement {
                attribute :>> origin = (0, 0, 0);
            }
        }
        
        attribute wheelPosition1 :> ISQ::cartesianPosition3dVector = (50, 0, 0) [vcf];
        
        part leftFrontWheel : Wheel[1] :> componentItems {
            attribute :>> coordinateFrame : CoordinateFramePlacement {
                attribute :>> origin = wheelPosition1;
                attribute :>> basisDirections; // = ...
            }
        }
        
        part rightFrontWheel : Wheel[1] :> componentItems {
            attribute :>> coordinateFrame : CoordinateFramePlacement {
                attribute :>> origin = wheelPosition1 + (0, 184, 0) [vcf];
                attribute :>> basisDirections; // = ...
           }
        }
        
        part leftRearWheel : Wheel[1] :> componentItems {
            attribute :>> coordinateFrame : CoordinateFramePlacement {
                attribute :>> origin = wheelPosition1 + (300, 0, 0) [vcf];
                attribute :>> basisDirections; // = ...
           }
        }
        
        part rightRearWheel : Wheel[1] :> componentItems {
            attribute :>> coordinateFrame : CoordinateFramePlacement {
                attribute :>> origin = wheelPosition1 + (300, 184, 0) [vcf];
                attribute :>> basisDirections; // = ...
           }
        }        
    }
}

Package VehicleGeometry (d6dfe2bf-c1c1-472a-9a73-6a1ba4a10893)


# Vehicle Motion

In [6]:
package VehicleMotion {
    import VehicleDefinitions::*;
    import SpatialItems::*;
    
    part referenceSpace : SpatialItem {
        
        part vehicle : Vehicle, SpatialItem {
            attribute mass :> ISQ::mass;
            attribute acceleration :> ISQ::cartesianAcceleration3dVector;
        }
        
        attribute startTime :> Time::timeInstant;
        derived attribute elapseTime :> ISQ::duration = localClock.currentTime - startTime;
        
        assert constraint constantAccelerationConstraint {
            CurrentPositionOf(vehicle.originPoint, referenceSpace) ==
                mass * acceleration * elapseTime ^ 2 / 2.0
        }
        
    }
}

Package VehicleMotion (43054bf2-eac0-4ef1-8fe4-957194260b27)
